# Conn Settings

In [1]:
import sqlite3
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
conn = sqlite3.connect("main.db")

# Import To JournalEntry_Rad

In [3]:
raw = pd.read_csv(r"C:\Users\adamss\OneDrive - Central Health\Budget Project\csvUploads\jt.csv")

In [4]:
raw.head()

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,BusinessUnit,Account,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,09/30/2020,Financial,NaN,NaN,...,HCD - Eligiblity & Enrollment,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,09/30/2020,Financial,NaN,NaN,...,LPPF - Local Provider Participation Fund,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
2,1.0,1921.0,8950.0,50000,"76,451.65",USD,09/30/2020,Financial,NaN,NaN,...,Section 1115 Waiver Costs,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
3,1.0,1921.0,9500.0,50000,-5.98,USD,09/30/2020,Financial,NaN,NaN,...,Finance,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
4,1.0,1921.0,4200.0,50000,"97,178.14",USD,09/30/2020,Financial,NaN,NaN,...,HCD - Service Delivery Operations,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN


In [5]:
df = raw.copy()
df = df.reset_index(drop=True)
df['Id'] = df.index + 1
df.head(2)

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,Account,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3,Id
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,09/30/2020,Financial,NaN,NaN,...,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,1
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,09/30/2020,Financial,NaN,NaN,...,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,2


In [6]:
# Function to determine fiscal year based on date
def get_fiscal_year(date):
    if pd.isna(date):
        return "UnkownFiscalYear"

    year = date.year
    
    # Define fiscal year start and end based on the current year
    fiscal_start = pd.Timestamp(f'{year-1}-10-01')
    fiscal_end = pd.Timestamp(f'{year}-09-30')

    # If date is between October 1st of the previous year and September 30th of the current year
    if fiscal_start <= date <= fiscal_end:
        return f'FY{str(year)[-2:]}'  # Current year as fiscal year
    else:
        return f'FY{str(year + 1)[-2:]}'  # Next year as fiscal year
    

In [7]:
df = raw.copy()
df.columns = df.columns.str.replace(" ", "")
df = df.reset_index(drop=True)
df["Id"] = df.index + 1
df["AccountingDate"] = pd.to_datetime(
    df["AccountingDate"], format="%m/%d/%Y", errors="coerce"
)
df["FiscalYear"] = df["AccountingDate"].apply(get_fiscal_year)
df.head(2)

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3,Id,FiscalYear
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,2020-09-30,Financial,NaN,NaN,...,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,1,FY20
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,2020-09-30,Financial,NaN,NaN,...,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,2,FY20


In [17]:
df_split = df.copy()
cols = [col for col in df_split.columns if "RAD" in col and "Description" not in col]

In [18]:

df_split[cols] = df_split[cols].applymap(lambda x: str(x) if not pd.isna(x) else np.nan)
df_split["Combined"] = df_split[cols].apply(
    lambda row: "|".join([str(x) for x in row if not pd.isna(x)]), axis=1
)
df_split = df_split.assign(value=df_split['Combined'].str.split('|')).explode('value')
df_result = df_split[['Id', 'value']].reset_index(drop=True)

In [27]:
df_result.replace("", float("NaN"), inplace=True)
df_result.dropna(inplace=True)
df_result.rename(columns={'value': 'RADId', 'Id': 'JournalEntryId'}, inplace=True)
df_result

,JournalEntryId,RADId
0,1,REG
1,2,REG
2,3,REG
3,4,REG
4,5,PTO
...,...,...
224530,210123,CH RZ
224531,210124,CH RZ
224534,210127,CELL
224544,210137,LONSTA


In [28]:
def import_func(df: pd.DataFrame()) -> None:
    df.to_sql('JournalEntry_Rad', conn, if_exists='append', index=False)
    
import_func(df_result)

# Import To JournalEntry

In [30]:
df = df[
    [
        "Id",
        "FiscalYear",
        "CompanyId",
        "EntryId",
        "BusinessUnitId",
        "AccountNo",
        "Amount",
        "AccountingDate",
        "Remarks",
    ]
]

In [31]:
def import_func(df: pd.DataFrame()) -> None:
    df.to_sql('JournalEntry', conn, if_exists='append', index=False)
    
import_func(df)